In [1]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from collections import defaultdict
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score


In [2]:
resnet_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  
                         std=[0.229, 0.224, 0.225])   
])

In [3]:
train_dataset_resnet = torchvision.datasets.CIFAR10(
    root="F:/CIFAR10_Project/data",
    train=True,
    transform=resnet_transform,
    download=True
)

test_dataset_resnet = torchvision.datasets.CIFAR10(
    root="F:/CIFAR10_Project/data",
    train=False,
    transform=resnet_transform,
    download=True
)

In [4]:
# I was having issues where Python used my CPU instead of my GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = ResNet18_Weights.DEFAULT
resnet = resnet18(weights=weights)
resnet.fc = nn.Identity()
resnet = resnet.to(device)
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
def extract_features(dataset, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    features = []
    labels = []
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(device)
            feats = resnet(imgs)
            features.append(feats.cpu())
            labels.append(lbls)
    features = torch.cat(features)
    labels = torch.cat(labels)
    return features, labels

In [6]:
train_features, train_labels = extract_features(train_dataset_resnet)
test_features, test_labels = extract_features(test_dataset_resnet)

In [7]:
def select_n_per_class(features, labels, n):
    selected_feats = []
    selected_labels = []
    counts = defaultdict(int)
    for feat, lbl in zip(features, labels):
        if counts[int(lbl)] < n:
            selected_feats.append(feat.unsqueeze(0))
            selected_labels.append(lbl.unsqueeze(0))
            counts[int(lbl)] += 1
        if all(c >= n for c in counts.values()):
            break
    selected_feats = torch.cat(selected_feats)
    selected_labels = torch.cat(selected_labels)
    return selected_feats, selected_labels

train_feats_500, train_lbls_500 = select_n_per_class(train_features, train_labels, 500)
test_feats_100, test_lbls_100 = select_n_per_class(test_features, test_labels, 100)

In [8]:
# Using PCA to reduce size
pca = PCA(n_components=50)
train_feats_pca = pca.fit_transform(train_feats_500.numpy())
test_feats_pca = pca.transform(test_feats_100.numpy())

In [9]:
# 1. Assign variables from your PCA features and labels
X_train = train_feats_pca
y_train = train_lbls_500.numpy()

X_test = test_feats_pca
y_test = test_lbls_100.numpy()

# training Gaussian Naive Bayes from scratch
import numpy as np

class GaussianNaiveBayesScratch:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.means = {}
        self.vars = {}
        self.priors = {}
        for c in self.classes:
            X_c = X[y == c]
            self.means[c] = X_c.mean(axis=0)
            self.vars[c] = X_c.var(axis=0) + 1e-9  # avoid zero variance
            self.priors[c] = len(X_c) / len(X)
    
    def _pdf(self, class_idx, x):
        mean = self.means[class_idx]
        var = self.vars[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
    
    def predict(self, X):
        y_pred = []
        for x in X:
            posteriors = []
            for c in self.classes:
                prior = np.log(self.priors[c])
                likelihoods = np.log(self._pdf(c, x) + 1e-9).sum()
                posterior = prior + likelihoods
                posteriors.append(posterior)
            y_pred.append(self.classes[np.argmax(posteriors)])
        return np.array(y_pred)

gnb_scratch = GaussianNaiveBayesScratch()
gnb_scratch.fit(X_train, y_train)
y_pred_scratch = gnb_scratch.predict(X_test)

# Scikit's Gaussian naive Bayes classifier
from sklearn.naive_bayes import GaussianNB

gnb_sklearn = GaussianNB()
gnb_sklearn.fit(X_train, y_train)
y_pred_sklearn = gnb_sklearn.predict(X_test)

# Evaluation using integrated metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def evaluate_model(y_true, y_pred):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='macro'))
    print("Recall :", recall_score(y_true, y_pred, average='macro'))
    print("F1-score :", f1_score(y_true, y_pred, average='macro'))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

print("Evaluation of Scratch GNB:")
evaluate_model(y_test, y_pred_scratch)

print("\nEvaluation of Sklearn GNB:")
evaluate_model(y_test, y_pred_sklearn)

Evaluation of Scratch GNB:
Accuracy: 0.791
Precision: 0.7962758408014589
Recall : 0.791
F1-score : 0.7919628823785148
Confusion Matrix:
 [[81  1  0  1  0  0  1  0 12  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 6  0 64  8  8  3 10  0  1  0]
 [ 1  0  4 73  4 11  6  1  0  0]
 [ 1  0  4  6 78  3  1  7  0  0]
 [ 0  1  5 15  3 73  2  1  0  0]
 [ 2  0  4  8  5  1 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 8  0  1  0  1  0  0  0 87  3]
 [ 5  4  0  2  0  0  0  1  1 87]]

Evaluation of Sklearn GNB:
Accuracy: 0.791
Precision: 0.7962758408014589
Recall : 0.791
F1-score : 0.7919628823785148
Confusion Matrix:
 [[81  1  0  1  0  0  1  0 12  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 6  0 64  8  8  3 10  0  1  0]
 [ 1  0  4 73  4 11  6  1  0  0]
 [ 1  0  4  6 78  3  1  7  0  0]
 [ 0  1  5 15  3 73  2  1  0  0]
 [ 2  0  4  8  5  1 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 8  0  1  0  1  0  0  0 87  3]
 [ 5  4  0  2  0  0  0  1  1 87]]


In [10]:
# Manual integration of metrics

def confusion_matrix_manual(y_true, y_pred, num_classes):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

def precision_recall_f1(cm):
    precisions = []
    recalls = []
    f1s = []
    for i in range(len(cm)):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    return np.mean(precisions), np.mean(recalls), np.mean(f1s)

def accuracy_manual(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def evaluate_manual(y_true, y_pred, num_classes):
    acc = accuracy_manual(y_true, y_pred)
    cm = confusion_matrix_manual(y_true, y_pred, num_classes)
    precision, recall, f1 = precision_recall_f1(cm)
    
    print("\nManual Evaluation:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print(f"F1-score (macro): {f1:.4f}")
    print("Confusion Matrix:\n", cm)


# Print results
print("Scratch GNB results:")
evaluate_manual(y_test, y_pred_scratch, num_classes=10)

print("\nSklearn GNB results:")
evaluate_manual(y_test, y_pred_sklearn, num_classes=10)

Scratch GNB results:

Manual Evaluation:
Accuracy: 0.7910
Precision (macro): 0.7963
Recall (macro): 0.7910
F1-score (macro): 0.7920
Confusion Matrix:
 [[81  1  0  1  0  0  1  0 12  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 6  0 64  8  8  3 10  0  1  0]
 [ 1  0  4 73  4 11  6  1  0  0]
 [ 1  0  4  6 78  3  1  7  0  0]
 [ 0  1  5 15  3 73  2  1  0  0]
 [ 2  0  4  8  5  1 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 8  0  1  0  1  0  0  0 87  3]
 [ 5  4  0  2  0  0  0  1  1 87]]

Sklearn GNB results:

Manual Evaluation:
Accuracy: 0.7910
Precision (macro): 0.7963
Recall (macro): 0.7910
F1-score (macro): 0.7920
Confusion Matrix:
 [[81  1  0  1  0  0  1  0 12  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 6  0 64  8  8  3 10  0  1  0]
 [ 1  0  4 73  4 11  6  1  0  0]
 [ 1  0  4  6 78  3  1  7  0  0]
 [ 0  1  5 15  3 73  2  1  0  0]
 [ 2  0  4  8  5  1 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 8  0  1  0  1  0  0  0 87  3]
 [ 5  4  0  2  0  0  0  1  1 87]]


In [11]:
# Save both variants

gnb_scratch.fit(X_train, y_train)
import pickle
with open('gnb_scratch.pkl', 'wb') as f:
    pickle.dump(gnb_scratch, f)

gnb_sklearn.fit(X_train, y_train)
import joblib
joblib.dump(gnb_sklearn, 'gnb_sklearn.pkl')

['gnb_sklearn.pkl']

In [12]:
# Load and test both variants from saved models

import pickle, joblib

# Load scratch
with open('gnb_scratch.pkl', 'rb') as f:
    gnb_scratch_loaded = pickle.load(f)

# Load sklearn
gnb_sklearn_loaded = joblib.load('gnb_sklearn.pkl')

# Predict after loading
y_pred_scratch_loaded = gnb_scratch_loaded.predict(X_test)
y_pred_sklearn_loaded = gnb_sklearn_loaded.predict(X_test)

# Evaluate
evaluate_model(y_test, y_pred_scratch_loaded)
evaluate_model(y_test, y_pred_sklearn_loaded)

Accuracy: 0.791
Precision: 0.7962758408014589
Recall : 0.791
F1-score : 0.7919628823785148
Confusion Matrix:
 [[81  1  0  1  0  0  1  0 12  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 6  0 64  8  8  3 10  0  1  0]
 [ 1  0  4 73  4 11  6  1  0  0]
 [ 1  0  4  6 78  3  1  7  0  0]
 [ 0  1  5 15  3 73  2  1  0  0]
 [ 2  0  4  8  5  1 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 8  0  1  0  1  0  0  0 87  3]
 [ 5  4  0  2  0  0  0  1  1 87]]
Accuracy: 0.791
Precision: 0.7962758408014589
Recall : 0.791
F1-score : 0.7919628823785148
Confusion Matrix:
 [[81  1  0  1  0  0  1  0 12  4]
 [ 3 88  0  2  1  0  0  0  0  6]
 [ 6  0 64  8  8  3 10  0  1  0]
 [ 1  0  4 73  4 11  6  1  0  0]
 [ 1  0  4  6 78  3  1  7  0  0]
 [ 0  1  5 15  3 73  2  1  0  0]
 [ 2  0  4  8  5  1 79  1  0  0]
 [ 1  1  0  5  6  5  0 81  1  0]
 [ 8  0  1  0  1  0  0  0 87  3]
 [ 5  4  0  2  0  0  0  1  1 87]]
